<a href="https://colab.research.google.com/github/UjjsBI/assignment/blob/main/DeepSeek_ollama_assignement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# Install lshw module so that Ollama will be able to see that
!sudo apt-get install lshw

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
lshw is already the newest version (02.19.git.2021.06.19.996aaad9c7-2build1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [29]:
!lshw

49200273ec6d
    description: Computer
    width: 64 bits
    capabilities: smp vsyscall32
  *-core
       description: Motherboard
       physical id: 0
     *-memory
          description: System memory
          physical id: 0
          size: 13GiB
     *-cpu
          product: Intel(R) Xeon(R) CPU @ 2.20GHz
          vendor: Intel Corp.
          physical id: 1
          bus info: cpu@0
          version: 6.79.0
          width: 64 bits
          capabilities: fpu fpu_exception wp vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp x86-64 constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
          configuratio

In [30]:
# Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [32]:
!ollama --version

ollama version is 0.6.2


In [33]:
# Run ollama serve in the backend, even if the above error is present, this will connect with GPU

import subprocess

# Run ollama serve in the background
process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Optionally, print the process ID
print(f"Ollama serve process ID: {process.pid}")

Ollama serve process ID: 16470


In [34]:
# Pull llama 1b parameter Model for testing
!ollama pull deepseek-r1:8b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 6340dc3229b0... 100% ▕▏ 4.9 GB                         
pulling 369ca498f347... 100% ▕▏  387 B                         
pulling 6e4c38e1172f... 100% ▕▏ 1.1 KB                         
pulling f4d24e9138dd... 100% ▕▏  148 B                         
pulling 0cb05c6e4e02... 100% ▕▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [39]:
# Sample program which runs and outputs in a readable format

import requests
import json

def query_ollama(prompt, model="deepseek-r1:8b"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }

    response = requests.post(url, json=payload)

    if response.status_code == 200:
        return response.json().get("response", "Error: No response from model")
    else:
        return f"Error: {response.status_code} - {response.text}"

# Sample customer-agent conversation
customer_query = "Hi, I received the wrong item in my order. How can I get a replacement?"
agent_prompt = f"You are an e-commerce customer support agent. Respond to this customer query professionally:\n{customer_query}"

agent_response = query_ollama(agent_prompt)

print("Customer:", customer_query)
print("Agent:", agent_response)



Customer: Hi, I received the wrong item in my order. How can I get a replacement?
Agent: <think>
Okay, so the user sent me a message saying they received the wrong item and want to know how to get a replacement. Hmm, first things first, I should acknowledge their issue quickly because that shows I'm paying attention and taking them seriously.

I need to make sure my response is clear and helpful. They probably feel frustrated or anxious about getting the wrong product, so I should keep the tone calm and professional. Let me outline what they might need: information on how to return the item, get a replacement, maybe a refund option if replacement isn't possible.

I should guide them through the process step by step without being too robotic. Maybe suggest reaching out via phone or email with their order details. It's important to encourage them to provide specific info so we can help faster.

Also, I should mention that someone will get back to them soon, which reassures them that thei

In [45]:
# Generate conversation summary and insights
summary_prompt = (
    "Here is a conversation between a customer and a support agent. "
    "Please provide a concise summary of the discussion and extract key insights about the customer's issue and concerns.\n\n"
    f"Customer: {customer_query}\n"
    f"Agent: {agent_response}\n"
)

summary_response = query_ollama(summary_prompt)

print("Full API Response:", summary_response)

print("\n\nCustomer:", customer_query)
print("\n\nAgent:", agent_response)
print("\n\nSummary and Insights:", summary_response)

Full API Response: 

**Customer Support Response:**

Dear [Customer's Name],

Thank you for informing us about the incorrect item received in your order. We apologize for the inconvenience and are here to help you resolve it promptly.

Please follow these steps:

1. **Reach Out:** Contact our support team via phone or email with your order number, delivery confirmation details, and information about the incorrect item.
2. **Provide Details:** Clearly describe the issue and include any relevant information that can assist us in identifying the correct product for replacement.
3. **Await Assistance:** A member of our team will review your request and respond within a short time with further instructions.

We aim to ensure this process is as smooth as possible and appreciate your patience while we work on it.

Looking forward to assisting you.

Best regards,  
[Your Name]  
Customer Support Team


Customer: Hi, I received the wrong item in my order. How can I get a replacement?


Agent: <